In [4]:
%pip install jupysql
%pip install mailbox

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [6]:
%load_ext sql

In [73]:
import re
import csv
import duckdb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%sql duckdb:///../datasets/email.db

In [8]:
import mailbox

mboxfile = "C:\\Users\\jmcar\\Downloads\\correos.mbox"
mbox = mailbox.mbox(mboxfile)

In [57]:
for key in mbox[0].keys():
    print(key)


X-GM-THRID
X-Gmail-Labels
Delivered-To
Received
X-Received
ARC-Seal
ARC-Message-Signature
ARC-Authentication-Results
Return-Path
Received
Received-SPF
Authentication-Results
DKIM-Signature
X-Google-DKIM-Signature
X-Gm-Message-State
X-Google-Smtp-Source
MIME-Version
X-Received
Date
Reply-To
X-Google-Id
Feedback-ID
X-Notifications
X-Notifications-Bounce-Info
Message-ID
Subject
From
To
Content-Type


2291

In [20]:
%%sql
create table if not exists mailbox (
    subject text,
    from_person text,
    date text,
    to_person text,
    labels text,
    gm_third text
)

,Count


In [58]:
%%sql

truncate mailbox

,Count
0,6586


In [60]:
con = duckdb.connect('../datasets/email.db')

for (key, message) in mbox.items():
    # print(type(message['subject']), type(message['from']), type(message['date']), type(message['to']), type(message['X-Gmail-Labels']), type(message['X-GM-THRID']))
    con.execute("insert into mailbox values (?, ?, ?, ?, ?, ?)", [message['subject'].__str__(), message['from'], message['date'], message['to'], message['X-Gmail-Labels'], message['X-GM-THRID']])


In [61]:
%%sql

select count(*) from mailbox

,count_star()
0,2291


In [62]:
%sql mailbox_df << select * from mailbox

In [68]:
mailbox_df['date'] = mailbox_df['date'].apply(lambda x: pd.to_datetime(x, errors='coerce', utc=True))

Timestamp('2023-09-28 01:55:17+0000', tz='UTC')

In [69]:
mailbox_df = mailbox_df[mailbox_df['date'].notna()]

In [70]:
mailbox_df.to_parquet('../datasets/email.parquet')

In [71]:
mailbox_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2290 entries, 0 to 2290
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   subject      2290 non-null   object             
 1   from_person  2290 non-null   object             
 2   date         2290 non-null   datetime64[ns, UTC]
 3   to_person    2265 non-null   object             
 4   labels       2290 non-null   object             
 5   gm_third     2290 non-null   object             
dtypes: datetime64[ns, UTC](1), object(5)
memory usage: 125.2+ KB


In [72]:
mailbox_df.head(10)

,subject,from_person,date,to_person,labels,gm_third
0,=?UTF-8?Q?Protege_tu_cuenta_de_administrador_d...,The Google Cloud Team <cloud-noreply@google.com>,2023-09-28 22:26:48+00:00,jm.carb@gmail.com,"=?UTF-8?Q?Importante,Papelera,Abierto,Categor=...",1778321991247361539
1,=?UTF-8?B?VHUgcGVkaWRvIGRlIEFtYXpvbi5jb20ubXgg...,"""Amazon.com.mx"" <shipment-tracking@amazon.com.mx>",2023-09-28 01:55:17+00:00,jm.carb@gmail.com,"=?UTF-8?Q?Categor=C3=ADa_Compras,Importante,Pa...",1778244510688553499
2,=?utf-8?Q?Important=20Update=3A=20Clarificatio...,=?utf-8?Q?Team=20Lens?= <noreply@k8slens.dev>,2023-09-27 20:58:27+00:00,<jm.carb@gmail.com>,"=?UTF-8?Q?Importante,Papelera,Abierto,Categor=...",1778225843025157777
3,Hurry! Your cart is about to expire!,Yahboom <market@yahboom.com>,2023-09-29 16:43:38+00:00,jm.carb@gmail.com,"=?UTF-8?Q?Importante,Papelera,Abierto,Categor=...",1778390998550442919
4,"ChatGPT Goes Multimodal, Dating Apps Embrace A...","""The Batch @ DeepLearning.AI"" <thebatch@deeple...",2023-09-28 18:02:22+00:00,jm.carb@gmail.com,"=?UTF-8?Q?Importante,Papelera,Abierto,Categor=...",1778305353971190954
5,=?UTF-8?B?RW50cmVnYWRvOiBUdSBwYXF1ZXRlIA==?=\n...,"""Amazon.com.mx"" <order-update@amazon.com.mx>",2023-10-01 00:27:59+00:00,jm.carb@gmail.com,"=?UTF-8?Q?Importante,Papelera,Abierto,Categor=...",1778483725064967777
6,=?UTF-8?B?RW50cmVnYWRvOiBUdSBwYXF1ZXRlIA==?=\n...,"""Amazon.com.mx"" <order-update@amazon.com.mx>",2023-09-30 17:17:30+00:00,jm.carb@gmail.com,"=?UTF-8?Q?Importante,Papelera,Abierto,Categor=...",1778483725064967777
7,=?UTF-8?Q?Welcome_to_Amazon_Assessments_=E2=80...,noreply@mail.amazon.jobs,2023-09-30 19:21:18+00:00,jm.carb@gmail.com,"=?UTF-8?Q?Importante,Papelera,Abierto,Categor=...",1778491514486185778
8,Re: Trail Cloud Account,Ashton Lafferty <support@nabucasa.com>,2023-06-05 05:33:55+00:00,jm.carb@gmail.com,"=?UTF-8?Q?Archivado,Importante,Abierto,Categor...",1767833959994718509
9,Re: Trail Cloud Account,Ashton Lafferty <support@nabucasa.com>,2023-06-05 04:04:02+00:00,jm.carb@gmail.com,"=?UTF-8?Q?Archivado,Importante,Abierto,Categor...",1767833959994718509


In [74]:
def extract_email_ID(string):
  email = re.findall(r'<(.+?)>', string)
  if not email:
    email = list(filter(lambda y: '@' in y, string.split()))
  return email[0] if email else np.nan

In [75]:
mailbox_df['from_person'] = mailbox_df['from_person'].apply(lambda x: extract_email_ID(x))

C:\Users\jmcar\AppData\Local\Temp\ipykernel_40244\3157214683.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mailbox_df['from_person'] = mailbox_df['from_person'].apply(lambda x: extract_email_ID(x))


In [76]:
mailbox_df.head(10)

,subject,from_person,date,to_person,labels,gm_third
0,=?UTF-8?Q?Protege_tu_cuenta_de_administrador_d...,cloud-noreply@google.com,2023-09-28 22:26:48+00:00,jm.carb@gmail.com,"=?UTF-8?Q?Importante,Papelera,Abierto,Categor=...",1778321991247361539
1,=?UTF-8?B?VHUgcGVkaWRvIGRlIEFtYXpvbi5jb20ubXgg...,shipment-tracking@amazon.com.mx,2023-09-28 01:55:17+00:00,jm.carb@gmail.com,"=?UTF-8?Q?Categor=C3=ADa_Compras,Importante,Pa...",1778244510688553499
2,=?utf-8?Q?Important=20Update=3A=20Clarificatio...,noreply@k8slens.dev,2023-09-27 20:58:27+00:00,<jm.carb@gmail.com>,"=?UTF-8?Q?Importante,Papelera,Abierto,Categor=...",1778225843025157777
3,Hurry! Your cart is about to expire!,market@yahboom.com,2023-09-29 16:43:38+00:00,jm.carb@gmail.com,"=?UTF-8?Q?Importante,Papelera,Abierto,Categor=...",1778390998550442919
4,"ChatGPT Goes Multimodal, Dating Apps Embrace A...",thebatch@deeplearning.ai,2023-09-28 18:02:22+00:00,jm.carb@gmail.com,"=?UTF-8?Q?Importante,Papelera,Abierto,Categor=...",1778305353971190954
5,=?UTF-8?B?RW50cmVnYWRvOiBUdSBwYXF1ZXRlIA==?=\n...,order-update@amazon.com.mx,2023-10-01 00:27:59+00:00,jm.carb@gmail.com,"=?UTF-8?Q?Importante,Papelera,Abierto,Categor=...",1778483725064967777
6,=?UTF-8?B?RW50cmVnYWRvOiBUdSBwYXF1ZXRlIA==?=\n...,order-update@amazon.com.mx,2023-09-30 17:17:30+00:00,jm.carb@gmail.com,"=?UTF-8?Q?Importante,Papelera,Abierto,Categor=...",1778483725064967777
7,=?UTF-8?Q?Welcome_to_Amazon_Assessments_=E2=80...,noreply@mail.amazon.jobs,2023-09-30 19:21:18+00:00,jm.carb@gmail.com,"=?UTF-8?Q?Importante,Papelera,Abierto,Categor=...",1778491514486185778
8,Re: Trail Cloud Account,support@nabucasa.com,2023-06-05 05:33:55+00:00,jm.carb@gmail.com,"=?UTF-8?Q?Archivado,Importante,Abierto,Categor...",1767833959994718509
9,Re: Trail Cloud Account,support@nabucasa.com,2023-06-05 04:04:02+00:00,jm.carb@gmail.com,"=?UTF-8?Q?Archivado,Importante,Abierto,Categor...",1767833959994718509
